In [21]:
import pandas as pd
import numpy as np

In [22]:
#type file names here
ua_file_name = 'UA_Source_Medium.csv'
g4_file_name = 'GA4_Source_Medium.csv'

In [23]:
#Read data, grab only the first table, not the date table
ua_file = pd.read_csv(ua_file_name, header = 6, thousands=',')
n_values = np.logical_not(ua_file.isnull()).sum(axis=1)
cut_off = n_values[n_values == 0].index[0]
ua_file = ua_file[:cut_off -1]
g4_file = pd.read_csv(g4_file_name, header = 6, thousands=',')
g4_file = g4_file[['Source / medium', 'Total users', 'Sessions']]
g4_file.rename(columns={'Source / medium': 'Source / Medium', 'Total users': 'Users'}, inplace = True)

#Run the code below (get rid of the hashtag) if the 'Users' or 'Sessions' columns have commas
ua_file['Users'] = ua_file['Users'].str.replace(',','')
# ua_file['Sessions'] = ua_file['Sessions'].str.replace(',','')
# g4_file['Users'] = g4_file['Users'].str.replace(',','')
# g4_file['Sessions'] = g4_file['Sessions'].str.replace(',','')

#Add data source column
ua_file['data_source'] = 'UA'
g4_file['data_source'] = 'G4'

#Append data, reset index
data = ua_file.append(g4_file).reset_index()
data.drop(columns={'index'}, inplace = True)
data

,Source / Medium,Users,Sessions,data_source
0,google / organic,93557,130831.0,UA
1,google / cpc,72195,102148.0,UA
2,(direct) / (none),51478,69490.0,UA
3,dfa / cpm,22189,35882.0,UA
4,numero-masse / video,17056,27396.0,UA
...,...,...,...,...
5674,yallakora.com / referral,1,1.0,G4
5675,yellow-search.com / referral,1,1.0,G4
5676,yellowpages.ca / referral,1,1.0,G4
5677,youtube_bc / olv,1,1.0,G4


In [24]:
#Create pivot table
piv = pd.pivot_table(data, values = ['Users', 'Sessions'], index = 'Source / Medium', columns = 'data_source', aggfunc = {'Users':sum, 'Sessions':sum})
piv.columns = [c[1] + ' ' +  c[0] for c in piv.columns]
piv['G4 Users'] = piv['G4 Users'].astype('float')
piv['UA Users'] = piv['UA Users'].astype('float')
piv['Session Difference'] = piv['G4 Sessions'] - piv['UA Sessions']
piv['Session Variance (%)'] = (piv['G4 Sessions'] - piv['UA Sessions'])/ piv['UA Sessions']
piv['Users Difference'] = piv['G4 Users'] - piv['UA Users']
piv['Users Variance (%)'] = (piv['G4 Users'] - piv['UA Users'])/ piv['UA Users']
piv = piv[[ 'G4 Users', 'UA Users', 'Users Difference', 'Users Variance (%)', 'G4 Sessions', 'UA Sessions', 'Session Difference', 'Session Variance (%)']]
piv.sort_values(by = ['G4 Users'], ascending = False, inplace = True)
piv

,G4 Users,UA Users,Users Difference,Users Variance (%),G4 Sessions,UA Sessions,Session Difference,Session Variance (%)
Source / Medium,,,,,,,,
(not set) / (not set),288665.0,2960.0,285705.0,96.521959,9887.0,3948.0,5939.0,1.504306
(direct) / (none),86465.0,51478.0,34987.0,0.679650,110605.0,69490.0,41115.0,0.591668
google / organic,83615.0,93557.0,-9942.0,-0.106267,120074.0,130831.0,-10757.0,-0.082221
google / cpc,44238.0,72195.0,-27957.0,-0.387243,60819.0,102148.0,-41329.0,-0.404599
Numero-Masse / Video,17586.0,NaN,NaN,NaN,26196.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
www-theweathernetwork-com.cdn.ampproject.org / referral,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN
zbexfgvatdybhe_fzbvy / gfkg_cevibdlcbyvdl,NaN,3.0,NaN,NaN,NaN,3.0,NaN,NaN
zbexfgvatdybhe_fzbvy / gfkg_gfezfbaedbaevgvbaf,NaN,21.0,NaN,NaN,NaN,21.0,NaN,NaN


In [20]:
#export to excel
piv.to_excel('Pivot_Source_Medium.xlsx')